In [7]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import types
from pyspark.sql import functions as F
from datetime import datetime

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/08 14:50:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [4]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [15]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [18]:
def prepare_for_grouping(row):
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)
    
    return (key, value)

In [20]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count

    return (output_amount, output_count)

In [25]:
from collections import namedtuple

In [26]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [27]:
def unwrap(row):
    return RevenueRow(
        hour = row[0][0], 
        zone= row[0][1], 
        revenue = row[1][0],
        count = row[1][1]
    )

In [31]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True), 
    types.StructField('zone', types.IntegerType(), True), 
    types.StructField('revenue', types.DoubleType(), True), 
    types.StructField('count', types.IntegerType(), True)]
    )

In [32]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema)

In [34]:
df_result.write.parquet('tmp/green-revenue')

In [37]:
columns = ['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [38]:
rows = duration_rdd.take(10)

In [40]:
#model = ...

def model_predict(df):
    # y_pred = model.predic(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [45]:
def apply_model_in_batch(partition):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions
    
    for row in df.itertuples():
        yield row

In [47]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch) \
    .toDF() \
    .drop('Index')

In [48]:
df_predicts.show()

+--------+--------------------+---------------------+------------+------------+-------------+------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|PULocationID|DOLocationID|trip_distance|predicted_duration|
+--------+--------------------+---------------------+------------+------------+-------------+------------------+
|     2.0|                  {}|                   {}|          74|         130|        12.77|63.849999999999994|
|     NaN|                  {}|                   {}|          67|          39|          8.0|              40.0|
|     2.0|                  {}|                   {}|         260|         157|         1.27|              6.35|
|     2.0|                  {}|                   {}|          82|          83|         1.25|              6.25|
|     2.0|                  {}|                   {}|         166|          42|         1.84| 9.200000000000001|
|     2.0|                  {}|                   {}|         179|         223|         0.76|   